In [1]:
import cv2

In [ ]:
degrees = ['15cm_0mm_0deg', '15cm_25mm_5deg', '15cm_50mm_10deg', '20cm_0mm_0deg', '20cm_25mm_5deg', '20cm_50mm_10deg']
ear_types = ['free', 'attached']

In [ ]:
for ear_type in ear_types:
    names = os.listdir(os.path.join(marked_img, ear_type, "001_original_data"))
    for name in names:
        for deg in degrees:
            marked_video = os.path.join(marked_img, ear_type, "001_original_data", name, deg, "resize.mp4")
            unmarked_video = os.path.join(unmarked_img, ear_type, "001_original_data", name, deg, "resize.mp4")
            count_0 = 0
            count_1 = 0
            
            
            marked_image_folder = os.path.join(marked_img, ear_type, "002_image", name, deg)
            if not os.path.exists(marked_image_folder):
                os.makedirs(marked_image_folder)
                
            unmarked_image_folder = os.path.join(unmarked_img, ear_type, "002_image", name, deg)
            if not os.path.exists(unmarked_image_folder):
                os.makedirs(unmarked_image_folder)
            
            cap = cv2.VideoCapture(marked_video)
            if not cap.isOpened():
                print("Cannot open camera")
                exit()
            while True:
                ret, frame = cap.read()
                
                if not ret:
                    print("Cannot receive frame")   
                    break
                    
                cv2.imwrite(os.path.join(marked_image_folder,"%04d.png"%count_0), frame)
#                 cv2.imshow('oxxostudio', frame) 
                count_0 += 1
            cap.release()                           
            
            cap = cv2.VideoCapture(unmarked_video)
            if not cap.isOpened():
                print("Cannot open camera")
                exit()
            while True:
                ret, frame = cap.read()             
                if not ret:
                    print("Cannot receive frame")   
                    break
                cv2.imwrite(os.path.join(unmarked_image_folder,"%04d.png"%count_1), frame)


                count_1 += 1
            cap.release()          
            
            print(deg)
            
            
            
            cv2.destroyAllWindows() 

In [1]:
import cv2
import numpy as np
import os
from mmdet.apis import init_detector, inference_detector
from mmdet.utils import register_all_modules
from mmdet.registry import VISUALIZERS
import mmcv
import json
import os

marked_img = "../feature_matching/marked"
unmarked_img = "../feature_matching/unmarked"
degrees = ['15cm_0mm_0deg', '15cm_25mm_5deg', '15cm_50mm_10deg', '20cm_0mm_0deg', '20cm_25mm_5deg', '20cm_50mm_10deg']
ear_types = ['free', 'attached']
keypoint = "../keypoint"


In [ ]:
MIN_MATCH_COUNT = 10
for ear_type in ear_types:
    names = os.listdir(os.path.join(marked_img, ear_type, "002_image"))
    for name in names:
        for deg in degrees:
            imgs = os.listdir(os.path.join(marked_img, ear_type, "002_image", name, deg))
            first_frame = mmcv.imread(os.path.join(marked_img, ear_type, "002_image", name, deg,imgs[0]), channel_order='rgb')
#             result_first_frame = inference_detector(model, first_frame)
#             bbox_first_frame = result_first_frame.pred_instances.bboxes[0]
            cv_img_first_frame = cv2.imread(os.path.join(marked_img, ear_type, "002_image", name, deg,imgs[0]))
#             ear = cv_img_first_frame[int(bbox_first_frame[1]):int(bbox_first_frame[3]), int(bbox_first_frame[0]):int(bbox_first_frame[2])].copy()

            
            json_file = os.path.join(keypoint, ear_type, "2_json", name, deg, "mask.json")
            with open(json_file, "r") as roi_f:
                rois = json.load(roi_f)
            
            roi_ = []
            for i in range(0, len(rois['frame_0'])):
                left_top = rois["frame_0"][str(i)]["left_top"]
                right_bot = rois["frame_0"][str(i)]["right_bot"]
                roi_.append((left_top[0], left_top[1], right_bot[0], right_bot[1]))
            

#             cv2.namedWindow('image')

#             cv2.createTrackbar('HMin', 'image', 0, 255, nothing)
#             cv2.createTrackbar('SMin', 'image', 0, 255, nothing)
#             cv2.createTrackbar('VMin', 'image', 0, 255, nothing)
#             cv2.createTrackbar('HMax', 'image', 0, 255, nothing)
#             cv2.createTrackbar('SMax', 'image', 0, 255, nothing)
#             cv2.createTrackbar('VMax', 'image', 0, 255, nothing)

            
#             cv2.setTrackbarPos('HMin', 'image', 0)
#             cv2.setTrackbarPos('SMin', 'image', 0)
#             cv2.setTrackbarPos('VMin', 'image', 0)
#             cv2.setTrackbarPos('HMax', 'image', 255)
#             cv2.setTrackbarPos('SMax', 'image', 255)
#             cv2.setTrackbarPos('VMax', 'image', 255)
#             lower = np.empty((1, 3))
#             upper = np.empty((1, 3))
            mask = np.zeros((960, 540), dtype=np.uint8)
           
#             while True:
                
#                 h_min = cv2.getTrackbarPos('HMin', 'image')
#                 s_min = cv2.getTrackbarPos('SMin', 'image')
#                 v_min = cv2.getTrackbarPos('VMin', 'image')
#                 h_max = cv2.getTrackbarPos('HMax', 'image')
#                 s_max = cv2.getTrackbarPos('SMax', 'image')
#                 v_max = cv2.getTrackbarPos('VMax', 'image')

                
#                 ff_hsv = cv2.cvtColor(ear, cv2.COLOR_BGR2HSV)

                # 设置HSV的阈值
#                 lower = np.array([h_min, s_min, v_min])
#                 upper = np.array([h_max, s_max, v_max])

                # 使用颜色范围进行过滤
                
                
            for i in range(0, len(roi_)):
                mask_ear = cv2.medianBlur(cv_img_first_frame[roi_[i][1]:roi_[i][3], roi_[i][0]:roi_[i][2]],5)
                mask_ear_gray = cv2.cvtColor(mask_ear, cv2.COLOR_BGR2GRAY)
                th6 = cv2.adaptiveThreshold(mask_ear_gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,7)
                
                final = np.ones((mask_ear.shape[0], mask_ear.shape[1]), dtype=np.uint8)*255 - th6
                mask[roi_[i][1]:roi_[i][3], roi_[i][0]:roi_[i][2]] = final
            
            
            kernel = np.ones((3,3), np.uint8)
            mask = cv2.dilate(mask, kernel, iterations=3)
            mask_ = np.ones((960, 540), dtype=np.uint8)*255-mask

                                      # 显示过滤后的图像
            filtered_image = cv2.bitwise_and(cv_img_first_frame, cv_img_first_frame, mask=mask_)
            
            kernel_ = np.ones((5,5), np.uint8)

            b, g, r = cv2.split(filtered_image)
            dilated_b = cv2.dilate(b, kernel_, iterations=3)
            dilated_g = cv2.dilate(g, kernel_, iterations=3)
            dilated_r = cv2.dilate(r, kernel_, iterations=3)
            dilated_image = cv2.merge((dilated_b, dilated_g, dilated_r))
            inpainting = cv2.bitwise_and(dilated_image, dilated_image, mask=mask)
                
#             filtered_image = cv2.medianBlur(filtered_image,5)
#             filtered_image_gray = cv2.cvtColor(filtered_image, cv2.COLOR_BGR2GRAY)
#             th6 = cv2.adaptiveThreshold(filtered_image_gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)

            
            dst_TELEA = cv2.inpaint(cv_img_first_frame,mask,3,cv2.INPAINT_TELEA)
            
            cv2.imshow('Image', dst_TELEA)
#             cv2.imshow('Filtered Image', filtered_image+inpainting)
#             cv2.waitKey(0)
#             cv2.destroyAllWindows()
            
#             
            
            
#             
            
            
#           
                
                
#                
                        
                        


                    
                    
